In [155]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize

In [156]:
#### CONSTANTES

In [157]:
PATH_BASE_DE_DADOS = 'data/noticias_estadao.csv'
VALOR_AXIS = 1
QUANTIDADE_ROWS = 10
TITULO_COL = "titulo"
CONTEUDO_COL = "conteudo"
NOTICIA_COMPLETA_COL = "noticia_completa"
ID_NOTICIA_COL = "idNoticia"
TOKENS_COL = "tokens"
TERMO_COL = "termo"
AND = "AND"
OR = "OR"

In [158]:
#### CLASSE

In [159]:
# Modela um indice invertido baseado no livro texto da disciplina (https://nlp.stanford.edu/IR-book/)
class IndiceInvertido:
    def __init__(self, termo, freq, posting_lists):
        self.termo = termo
        self.freq = freq
        self.posting_lists = posting_lists
        
    def get_termo(self):
        return self.termo
    
    def get_freq(self):
        return self.freq
    
    def get_posting_lists(self):
        return self.posting_lists

In [160]:
#### MÉTODOS

In [161]:
# concatena titulo com conteudo da noticia afim de descartar uma coluna
def concatena_titulo_conteudo(row):
    return (row[TITULO_COL] + " " + row[CONTEUDO_COL]).lower()

In [162]:
# tokeniza todo o conteudo de uma sentenca
def tokeniza(conteudo):
    return set(word_tokenize(conteudo))

In [163]:
# associa um token para cada id de noticia (one-to-one)
def associa_token_noticia(lista_tokens_associados, row):
    for token in row[TOKENS_COL]:
        lista_tokens_associados.append({
            TERMO_COL: token.strip(),
            ID_NOTICIA_COL: row[ID_NOTICIA_COL]
        })

In [164]:
# Cria indice invertido a partir da classe definida anteriormente
def cria_indice_invertido(tokens_associados_df):
    indice_invertido = dict()
    for termo, itens_agrupados in tokens_associados_df.groupby([TERMO_COL]):
        freq = len(itens_agrupados.get_values())
        posting_lists = set(itens_agrupados[ID_NOTICIA_COL])
        indice_invertido[termo] = IndiceInvertido(termo, freq, posting_lists)
    return indice_invertido

In [165]:
# verifica se query tem apenas 1 termo
def tem_um_termo(query):  
    return len(query.split(" ")) == 1

In [166]:
# ordena os termos do indice invertido por frequencia
def ordena_por_frequencia(termos, indice_invertido):
    resultado = list(map(lambda termo: indice_invertido[termo], termos))
    resultado.sort(key = lambda termo: termo.get_freq())
    return resultado

In [167]:
# recupera conjuncao que pode ser AND ou OR
def get_conjuncao(query):  
    return AND if AND in query else OR

In [168]:
def busca_booleana(termos, conjuncao, indice_invertido):
    ordenados_por_frequencia = ordena_por_frequencia(termos, indice_invertido)
    posting_lists = ordenados_por_frequencia[0].get_posting_lists()
    resultado = posting_lists
    
    for termo in ordenados_por_frequencia:
        posting_lists = termo.get_posting_lists()  
        if conjuncao == AND:
            resultado = resultado & posting_lists
        elif conjuncao == OR:
            resultado = resultado | posting_lists
            
    return list(resultado)

In [169]:
def busca(query, indice_invertido):
    if tem_um_termo(query):
        termo = query.lower()
        return indice_invertido[termo].get_posting_lists()
    else:
        conjuncao = get_conjuncao(query)
        termos = query.split(" " + conjuncao + " ")
        termos = list(map(lambda termo:termo.lower(), termos))
        return busca_booleana(termos, conjuncao, indice_invertido)

In [170]:
#### WORKFLOW

In [185]:
# importando base de dados no formato do DataFrame
df = pd.read_csv(PATH_BASE_DE_DADOS)

In [186]:
# mostra os 10 (QUANTIDADE_ROWS) primeiros itens da DataFrame
df.head(n=QUANTIDADE_ROWS)

,titulo,conteudo,idNoticia
0,11 dos eleitores do País são filiados a legendas,Há porém variações regionais nesse fenômeno En...,7617
1,11 executivos integram 1º pedido de condenação...,CURITIBA A força-tarefa da Operação Lava Jato ...,412
2,11 executivos integram 1º pedido de condenação...,CURITIBA A força-tarefa da Operação Lava Jato ...,415
3,13 de deputados do PMDB quer romper com PT,O Estado ouviu 54 dos 74 deputados do PMDB em ...,6736
4,2014 começou em 2007,O estudo do Estadão Dados publicado ontem sobr...,7611
5,2014 passa pelos palanques regionais diz Lupi,Segundo ele há uma falta de acordo na composiç...,7423
6,2018 com jeito de 2006,O resultado da eleição com o triunfo da polari...,1004
7,2º mandato de Dilma será o mais dependente do ...,O segundo mandato da presidente Dilma Rousseff...,71
8,2º turno se discute no 2º turno defende Marina,A ex-senadora tem sido alvo de críticas de gru...,3125
9,360 graus,Na batalha da comunicação conseguiu uma proeza...,4708


In [187]:
# aplica a função de concatenação em todas as linhas do DataFrame
df[NOTICIA_COMPLETA_COL] = df.apply(lambda row: concatena_titulo_conteudo(row), axis=VALOR_AXIS)

In [188]:
# remove colunas desnecessárias
df = df[[ID_NOTICIA_COL, NOTICIA_COMPLETA_COL]]

In [189]:
df.head(n=QUANTIDADE_ROWS)

,idNoticia,noticia_completa
0,7617,11 dos eleitores do país são filiados a legend...
1,412,11 executivos integram 1º pedido de condenação...
2,415,11 executivos integram 1º pedido de condenação...
3,6736,13 de deputados do pmdb quer romper com pt o e...
4,7611,2014 começou em 2007 o estudo do estadão dados...
5,7423,2014 passa pelos palanques regionais diz lupi ...
6,1004,2018 com jeito de 2006 o resultado da eleição ...
7,71,2º mandato de dilma será o mais dependente do ...
8,3125,2º turno se discute no 2º turno defende marina...
9,4708,360 graus na batalha da comunicação conseguiu ...


In [190]:
# aplica a funcao de tokeniza em todas as rows
df[TOKENS_COL] = df.apply(lambda row: tokeniza(row[NOTICIA_COMPLETA_COL]), axis=VALOR_AXIS)

In [191]:
df.head(n=QUANTIDADE_ROWS)

,idNoticia,noticia_completa,tokens
0,7617,11 dos eleitores do país são filiados a legend...,"{agosto, quem, no, amazonas, busca, sul, há, h..."
1,412,11 executivos integram 1º pedido de condenação...,"{ordens, tinham, no, organização, usaram, açõe..."
2,415,11 executivos integram 1º pedido de condenação...,"{tinham, no, organização, usaram, ações, setal..."
3,6736,13 de deputados do pmdb quer romper com pt o e...,"{entrevistas, no, mereceria, silva, sobre, é, ..."
4,7611,2014 começou em 2007 o estudo do estadão dados...,"{precisam, quem, passaram, no, pleito, partidá..."
5,7423,2014 passa pelos palanques regionais diz lupi ...,"{sendo, segundo, no, pdt, como, a, reunir, def..."
6,1004,2018 com jeito de 2006 o resultado da eleição ...,"{geraldo, mandato, atual, relação, derrubar, n..."
7,71,2º mandato de dilma será o mais dependente do ...,"{rebelião, mandato, começará, controlará, no, ..."
8,3125,2º turno se discute no 2º turno defende marina...,"{brasileiros, iniciativas, quem, atual, ponto,..."
9,4708,360 graus na batalha da comunicação conseguiu ...,"{mário, barbacena, quem, derrubar, no, assunto..."


In [192]:
lista_tokens_associados = []

In [193]:
df.apply(lambda row: associa_token_noticia(lista_tokens_associados, row), axis=VALOR_AXIS)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
7613    None
7614    None
7615    None
7616    None
7617    None
7618    None
7619    None
7620    None
7621    None
7622    None
7623    None
7624    None
7625    None
7626    None
7627    None
7628    None
7629    None
7630    None
7631    None
7632    None
7633    None
7634    None
7635    None
7636    None
7637    None
7638    None
7639    None
7640    None
7641    None
7642    None
Length: 7643, dtype: object

In [194]:
print(lista_tokens_associados[:QUANTIDADE_ROWS])

[{'termo': 'agosto', 'idNoticia': 7617}, {'termo': 'quem', 'idNoticia': 7617}, {'termo': 'no', 'idNoticia': 7617}, {'termo': 'amazonas', 'idNoticia': 7617}, {'termo': 'busca', 'idNoticia': 7617}, {'termo': 'sul', 'idNoticia': 7617}, {'termo': 'há', 'idNoticia': 7617}, {'termo': 'habitantes', 'idNoticia': 7617}, {'termo': 'regionais', 'idNoticia': 7617}, {'termo': 'é', 'idNoticia': 7617}]


In [195]:
tokens_associados_df = pd.DataFrame(lista_tokens_associados)

In [196]:
tokens_associados_df.head(n=QUANTIDADE_ROWS)

,idNoticia,termo
0,7617,agosto
1,7617,quem
2,7617,no
3,7617,amazonas
4,7617,busca
5,7617,sul
6,7617,há
7,7617,habitantes
8,7617,regionais
9,7617,é


In [197]:
indice_invertido = cria_indice_invertido(tokens_associados_df)

In [202]:
# Exemplo dado no canvas
assert sorted(busca("Campina AND Grande", indice_invertido)) == sorted([1952, 4802, 1987, 6694, 5382, 1770, 2763, 1068, 5870, 2777, 1370, 2779])

# 1. debate, presidenciável (AND e OR)
assert len(busca("debate OR presidencial", indice_invertido)) == 1770
assert len(busca("debate AND presidencial", indice_invertido)) == 201

# 2. presidenciáveis, corruptos (AND e OR)
assert len(busca("presidenciáveis OR corruptos", indice_invertido)) == 164
assert len(busca("presidenciáveis AND corruptos", indice_invertido)) == 0

# 3. Belo, Horizonte (AND e OR)
assert len(busca("Belo OR Horizonte", indice_invertido)) == 331
assert len(busca("Belo AND Horizonte", indice_invertido)) == 242